In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("data/logs/events_polarization_notebook.log"),
#         logging.StreamHandler()
#     ]
# )

In [3]:
import seaborn as sns

sns.set_theme()
from load.utils import load_users
from preprocessing.utils import (
    load_event_comments,
    load_event_vocab,
)
from preprocessing.constants import (
    EVENTS_INFO,
)


In [4]:
EVENT_KEY = "us_elections_2016"
EVENT_DETAILS = EVENTS_INFO[EVENT_KEY]

logging.info(EVENT_DETAILS)

In [5]:
users = load_users(engine="polars")

In [6]:
users

,author,gender,angry,anti,astro,dangerous,doom,military,nobility,trump
0,7W7I3,,False,False,False,False,False,False,False,False
1,Iz5YE,,False,False,False,False,False,False,False,False
2,bOU3y,,False,False,False,False,False,False,False,False
3,KZypX,,False,False,False,False,False,False,False,False
4,WAaa9,,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
3649198,xwPfs,,False,False,False,False,False,False,False,False
3649199,qhZjo,,False,False,False,False,False,False,False,False
3649200,sU7A1,,False,False,False,False,False,False,False,False
3649201,HrRwG,,False,False,False,False,False,False,False,False


In [7]:
event_comments = load_event_comments(event_name=EVENT_KEY, backend="pandas")
event_vocab = load_event_vocab(event_name=EVENT_KEY)

In [8]:
event_authors = event_comments[["author", "party"]].drop_duplicates(subset=["author"])


In [9]:
event_author_features = users.merge(
    event_authors["author"],
    how="inner",
    left_on="author",
    right_on="author",
)

event_author_features = event_author_features.set_index("author")

In [10]:
event_author_features

,gender,angry,anti,astro,dangerous,doom,military,nobility,trump
author,,,,,,,,,
bOU3y,,False,False,False,False,False,False,False,False
KZypX,,False,False,False,False,False,False,False,False
71fZe,,False,False,False,False,False,False,False,False
8ZPRw,,False,False,False,False,False,False,False,False
8rkpX,,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
JUjQA,,False,False,False,False,False,False,False,False
DRVvg,,False,False,False,False,False,False,False,False
m0CxP,,False,False,False,False,False,False,False,False


In [11]:
event_author_features["gender"] = event_author_features["gender"].astype("str").replace(
    {"": 0, "f": 1, "m": 2},
).astype("int64[pyarrow]")


In [12]:
X = event_author_features.to_numpy(dtype=int)

In [13]:
author_party = users[["author"]].merge(
    event_authors,
    how="inner",
    left_on="author",
    right_on="author",
)

In [14]:
is_republican = (author_party["party"] == "rep").to_numpy(dtype=int)